<a href="https://colab.research.google.com/github/zinakhazaee/MachineLearning-SupervisedLearning_Regression/blob/main/house_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
import numpy as np
import pandas as pd
from IPython.display import display
from pandas.api.types import CategoricalDtype
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import mutual_info_regression
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Mute warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/kaggle/home-data-for-ml-course/train.csv', index_col=0)
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/kaggle/home-data-for-ml-course/test.csv', index_col=0)
test.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [ ]:
train.shape

(1460, 80)

In [ ]:
train_null = pd.DataFrame(train.isnull().sum().sort_values(ascending=False))
train_null = train_null[train_null[0]>0]
train_null

,0
PoolQC,1453
MiscFeature,1406
Alley,1369
Fence,1179
FireplaceQu,690
LotFrontage,259
GarageYrBlt,81
GarageCond,81
GarageType,81
GarageFinish,81


In [ ]:
test.shape

(1459, 79)

In [ ]:
test_null = pd.DataFrame(test.isnull().sum().sort_values(ascending=False))
test_null = test_null[test_null[0]>0]
test_null

,0
PoolQC,1456
MiscFeature,1408
Alley,1352
Fence,1169
FireplaceQu,730
LotFrontage,227
GarageYrBlt,78
GarageFinish,78
GarageQual,78
GarageCond,78


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [ ]:
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


def plot_mi_scores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

In [ ]:
for name in train.select_dtypes("number"):
    train[name] = train[name].fillna(0)
for name in train.select_dtypes("object"):
    train[name] = train[name].fillna("None")

In [ ]:
for name in test.select_dtypes("number"):
    test[name] = test[name].fillna(0)
for name in test.select_dtypes("object"):
    test[name] = test[name].fillna("None")

In [ ]:
test.isnull().sum().sort_values(ascending=False)

MSSubClass       0
BedroomAbvGr     0
GarageType       0
FireplaceQu      0
Fireplaces       0
                ..
MasVnrArea       0
MasVnrType       0
Exterior2nd      0
Exterior1st      0
SaleCondition    0
Length: 79, dtype: int64

In [ ]:
y = np.log(train.SalePrice)

In [ ]:
X = train.drop('SalePrice', axis=1)

In [ ]:
X = (X - X.mean()) / X.std()

In [ ]:
test = (test - test.mean()) / test.std()

In [ ]:
mi_scores = make_mi_scores(X, y)
mi_scores = mi_scores[mi_scores>0.1]
print(mi_scores)
print(mi_scores.index)

OverallQual     0.567470
GrLivArea       0.459086
GarageCars      0.364223
GarageArea      0.362450
TotalBsmtSF     0.360716
YearBuilt       0.358423
1stFlrSF        0.299901
MSSubClass      0.272414
FullBath        0.261401
GarageYrBlt     0.258458
YearRemodAdd    0.233189
TotRmsAbvGrd    0.219780
LotFrontage     0.198691
2ndFlrSF        0.194431
LotArea         0.167469
Fireplaces      0.163421
OpenPorchSF     0.161249
BsmtFinSF1      0.143609
BsmtUnfSF       0.128090
OverallCond     0.112515
WoodDeckSF      0.105209
Name: MI Scores, dtype: float64
Index(['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF',
       'YearBuilt', '1stFlrSF', 'MSSubClass', 'FullBath', 'GarageYrBlt',
       'YearRemodAdd', 'TotRmsAbvGrd', 'LotFrontage', '2ndFlrSF', 'LotArea',
       'Fireplaces', 'OpenPorchSF', 'BsmtFinSF1', 'BsmtUnfSF', 'OverallCond',
       'WoodDeckSF'],
      dtype='object')


In [ ]:
features = ['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF',
       'YearBuilt', '1stFlrSF', 'MSSubClass', 'FullBath', 'GarageYrBlt',
       'YearRemodAdd', 'TotRmsAbvGrd', 'LotFrontage', '2ndFlrSF', 'LotArea',
       'Fireplaces', 'OpenPorchSF', 'BsmtFinSF1', 'BsmtUnfSF', 'OverallCond',
       'WoodDeckSF']

In [ ]:
X = X[features]
test = test[features]

In [ ]:
def rmse(actual, predict):
    return mean_squared_error(actual, predict, squared=False)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
params = dict(
    learning_rate=0.05,
    n_estimators=800,
)

xgb_model = XGBRegressor(**params)
xgb_model.fit(X_train, y_train,
             early_stopping_rounds=10,
             eval_set=[(X_valid, y_valid)],
             verbose=False)
xgb_pred = xgb_model.predict(X_valid)
rmse(y_valid, xgb_pred)

0.12835838129550042

In [ ]:
input = [('scale', StandardScaler()), ('polynomial', PolynomialFeatures(include_bias=False)), ('model', XGBRegressor(n_estimators=500, learning_rate=0.05))]
pipe = Pipeline(input)
pipe.fit(X_train, y_train)
pipe_pred = pipe.predict(X_valid)
rmse(y_valid, pipe_pred)

0.1323971368011147

In [ ]:
dt_model = DecisionTreeRegressor(random_state=0)
dt_model.fit(X_train, y_train)
dt_pred = dt_model.predict(X_valid)
rmse(y_valid, dt_pred)

0.20971664345013188

In [ ]:
rf_model = RandomForestRegressor(n_estimators=500, max_depth=25, random_state=0)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_valid)
rmse(y_valid, rf_pred)

0.13716081565681706

In [ ]:
test['SalePrice'] = np.exp(xgb_model.predict(test))

In [ ]:
output = pd.DataFrame({'Id': test.index, 'SalePrice': test.SalePrice})
output.to_csv('my_submission2.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
